In [1]:
import sys
sys.path.append("./defect_atom_diag")

In [2]:
import numpy as np
import pandas as pd
from functools import lru_cache as cache
from apply_sym_wan import make_ylm_wanns

In [38]:
N_MESH: int = 500

In [39]:
def normalize(A, delr):
    return A / np.sqrt(integrate(np.conjugate(A)*A, delr))

@cache
def ylm_wrapper(ll, rad_scale):
    wann, delr = make_ylm_wanns(ll, (N_MESH,)*3, "spherical", rad_scale)
    
    for i in range(len(wann)):
        if not np.all(np.isreal(wann[i])):
            wann[i] = normalize(wann[i], delr)
    
    return wann, delr

In [40]:
def integrate(A, delr):
    """
    Take the numerical integration (using summation approximation)
    
    Parameters:
        A (3d array): matrix of values to be integrated
        delr (list[float]): (dx, dy, dz)
    """
#     return np.sum(A) * np.product(delr)
#     print(delr)
    return np.trapz(np.trapz(np.trapz(A, dx=delr[0], axis=0), 
                             dx=delr[1], axis=0), dx=delr[2], axis=0)


def avg_ang_mom(psi, h_bar=1):
    """
    Take the average of angular momentum using wave function.
    
    Parameters:
        psi (3d array): wave function
        delr (list[float]): (dx, dy, dz)
        h_bar (float): value of reduced planck constant. Uses 1 by default.
    """
    # check if psi is real:
#     if np.all(np.isreal(psi)):
#         raise ValueError("psi is a real matrix")
    
    # construct axes:
    x = np.linspace(-1, 1, psi.shape[0])
    y = np.linspace(-1, 1, psi.shape[1])
    z = np.linspace(-1, 1, psi.shape[2])
    delr = (x[1]-x[0], y[1]-y[0], z[1]-z[0])
    xx, yy, zz = np.meshgrid(x, y, z, indexing="ij")
    
    if psi.shape != xx.shape:
        raise ValueError("Dimension mismatch")
        
    # complex conjugate of wave function:
    psi_conj = np.conjugate(psi)
    
    # partial derivatives:
    dpsi_dx, dpsi_dy, dpsi_dz = np.gradient(psi, *delr)
    
    # value of operator in each dimension:
    const = -1j * h_bar  # constant (-i*h_bar)
    L_x_op = const * (yy * dpsi_dz - zz * dpsi_dy)
    L_y_op = const * (zz * dpsi_dx - xx * dpsi_dz)
    L_z_op = const * (xx * dpsi_dy - yy * dpsi_dx)
    
    # average in each dimension
    L_x = np.real(integrate(psi_conj * L_x_op, delr))
    L_y = np.real(integrate(psi_conj * L_y_op, delr))
    L_z = np.real(integrate(psi_conj * L_z_op, delr))
    
    return L_x, L_y, L_z

In [41]:
def create_table(ll_range=[0, 3], rad_scale=0.01, h_bar=1):
    min_l = ll_range[0]
    max_l = ll_range[-1]

    col_headers = [f"m={m}" for m in range(-max_l, max_l + 1)]
    row_headers = [f"l={l}" for l in range(min_l, max_l + 1)]

    data = []
    
    for l in range(min_l, max_l + 1):
        psi = ylm_wrapper(l, rad_scale)[0]
        
        row = []
        
        i = 0
        for m in range(-max_l, max_l + 1):
            if -l <= m <= l:
#                 Lx, Ly, Lz = 
#                 print((Lx, Ly, Lz))
                # Store the vector as a tuple
                row.append(tuple("{:.2e}".format(value) \
                                 for value in avg_ang_mom(psi[i], h_bar)))
                i += 1
            else:
                row.append("-")
            
        data.append(row)

    df = pd.DataFrame(data, index=row_headers, columns=col_headers)
    return df

In [42]:
df = create_table(ll_range=[0, 1], h_bar=1)

In [43]:
df

,m=-1,m=0,m=1
l=0,-,"(0.00e+00, 0.00e+00, 0.00e+00)",-
l=1,"(-1.12e-21, 6.75e-19, -1.01e+00)","(0.00e+00, 0.00e+00, 0.00e+00)","(1.12e-21, -6.75e-19, 1.01e+00)"


In [37]:
wann, delr = ylm_wrapper(1, 0.01)
psi = wann[0]
integrate(np.conjugate(psi)*psi, delr)

(1+0j)